# Tabular Playground Competition
https://www.kaggle.com/c/tabular-playground-series-jan-2021

### Load dataset

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

In [96]:
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.io as pio

# these two lines are what allow your code to show up in a notebook
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

In [5]:
path='/Users/irak/Documents/py_viz/tabular-playground-series-jan-2021/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [41]:
train = pd.read_csv(path+'train.csv', sep=',', index_col='id')
sample_submission=pd.read_csv(path+'sample_submission.csv', sep=',', index_col='id')
test = pd.read_csv(path+'test.csv', sep=',', index_col='id')

print('train data')
display(train.head())
# print('sample sub')
# display(sample_submission.head())
# print('test data')
# display(test.head())


### На кагле написано ,что это просто датасет для обучения, у данных признаков нет никаких целей и описания, как и у таргета.
Над чем можно подумать:
1. Распределения признаков
2. Корреляции с целевой меткой
3. Можно посмотреть на выборсы боксплотом

На этом пока что идеи закончились(

# Визуализация

Корреляция между всеми переменными

In [81]:
corr = train.corr()
trace = go.Heatmap(z=corr.values,
                  x=corr.index.values,
                  y=corr.columns.values)
data = [trace]
fig = go.Figure(data = data)
fig.show(config = {'staticPlot': True})

In [40]:
print(train['target'].median())
print(train['target'].mean())

7.940570670373573
7.905661328316826


Распределение зависимой переменной

In [82]:
fig = px.histogram(x = train['target'])
fig.add_vline(train['target'].median(), line_color="orange", annotation_text="median 7.94")
fig.show(config = {'staticPlot': True})

Посмотрим на все переменные одновременно

In [83]:
fig_full = make_subplots(
    rows=7, cols=2,
    subplot_titles=('cont1', 'cont8', 'cont2', 'cont9', 'cont3', 'cont10', 'cont4', 'cont11', 'cont5', 'cont12', 'cont6', 'cont13', 'cont7', 'cont14'))

n = 1
for c in train.iloc[:,:7].columns:
    fig_full.add_trace(go.Histogram(x = train[c].values), row = n, col = 1)
    n+=1

k = 1
for p in train.iloc[:,7:14].columns:
    fig_full.add_trace(go.Histogram(x = train[p].values), row = k, col = 2)
    k+=1
    

fig_full.update_layout(showlegend=False, height = 2000)
fig_full.update_xaxes(matches='x')
fig_full.update_yaxes(matches='y')
fig_full.show(config = {'staticPlot': True})

Некоторые из распределений сильно отличаются от большинства - cont2, cont3, cont5. Они смещают общую для всех графиков шкалу Y, поэтому их придётся отделить, чтобы увидеть распределения с меньшими пиками.

In [46]:
fig_grid = make_subplots(
    rows=11, cols=1,
    subplot_titles=("cont1", "cont4", 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14'))

n = 1
for c in [train.columns[0], train.columns[3]] + train.columns[5:14].tolist():
    fig_grid.add_trace(go.Histogram(x = train[c].values), row = n, col = 1)
    n+=1

fig_grid.update_layout(showlegend=False, height = 2000)
fig_grid.update_xaxes(matches='x')
fig_grid.update_yaxes(matches='y')
fig_grid.show(config = {'staticPlot': True})

In [92]:
train.iloc[:,1]

Посмотрим на оставшиеся распределения отдельно

In [47]:
fig_grid2 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("cont2", "cont3"))

fig_grid2.add_trace(go.Histogram(x = train.iloc[:,1]), row = 1, col = 1)
fig_grid2.add_trace(go.Histogram(x = train.iloc[:,2]), row = 2, col = 1)

fig_grid2.update_layout(showlegend=False)
fig_grid2.update_xaxes(matches='x')
fig_grid2.update_yaxes(matches='y')
fig_grid2.show(config = {'staticPlot': True})

In [99]:
fig3 = px.histogram(x = train.iloc[:,4])
# fig3.show(config = {'staticPlot': True})
cont5_img = pio.write_image(fig3, 'cont5.png')
Image(cont5)

Посмотрим на выбросы

In [49]:
fig = px.box(boxmode="overlay")
for col in train.iloc[:,:14]:
    fig.add_trace(go.Box(y=train[col].values, 
                         name=train[col].name, 
                         marker_color='blue',
                         marker=dict(
                            outliercolor='orange',
                            line=dict(
                                outliercolor='darkblue',
                                outlierwidth=1))
                        )
                 )

fig.update_layout(showlegend=False)
fig.show(config = {'staticPlot': True})